In [1]:
#Imports:
from ncps.wirings import AutoNCP
from ncps.torch import CfC
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger
from numpy import genfromtxt
import numpy as np
import torch
import torch.utils.data as data
import matplotlib as plt
import torch.nn as nn

torch.set_float32_matmul_precision("high")

In [2]:
# Load Base File
#Current Simulation File
dataFile = 'Data/CfCMultiExtension/DoS_0709.csv'

dataSet = genfromtxt(dataFile, delimiter=',')

batchSize = 64
# Ceate dataloader and fill with (BSM, attk#). Expanding to add 0th dimension for batches.
# Batch size should be 64 for the low density simulations and 128 for high density simulations.
# No shuffle to keep batches on same vehicle.
# Num_workers is set to = num CPU cores
dataSet[0:-1,:] = dataSet[1:,:] # Get rid of the first null value of the dataset
# Sort Dataset by reciever id to pass on to every car.
dataSet = dataSet[np.argsort(dataSet[:, 1])]
print(dataSet.shape)
# count subsets per vehicle
unq, counts = np.unique(dataSet[:, 1], return_counts = True)
recvr = 0
lastRecieverCount = 0
newData = []
# Organize dataset into sets of 10 messages by reciever
while recvr < counts.shape[0]:
    # Loop through reciever
    index = 0
    while index < counts[recvr] - 10:
        # Loop through messages from reciever
        newData.append(dataSet[lastRecieverCount+index:lastRecieverCount +index+10])
        index += 5
    recvr += 1
    lastRecieverCount = counts[recvr-1]
dataSet = torch.tensor(newData)

(4957201, 12)


C:\Users\will\AppData\Local\Temp\ipykernel_19308\565797722.py:31: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  dataSet = torch.tensor(newData)


In [3]:
#PROPER FORMATTING
#Time sequences are 10 timepoints (Messages) with 7 features per message.
#Organized by car.

len = dataSet.shape[0]
trainPerc = 80
# Create new arrays per vehicle for federated learning
splits = np.split(dataSet, np.cumsum(counts)[:-1])
# Create seperate datasets for testing and training, using Train Percentage as metric for split
trainDataIn = torch.Tensor(dataSet[:int(len*(trainPerc/100)),:,1:10]).float()
trainDataOut = torch.Tensor(np.int_(dataSet[:int(len*(trainPerc/100)),:,11])).long()
testDataIn = torch.Tensor(dataSet[int(len*(trainPerc/100)):,:,1:10]).float()
testDataOut = torch.Tensor(np.int_(dataSet[int(len*(trainPerc/100)):,:,11])).long()
newsetIn = []
newsetOut = []
testsetIn = []
testsetOut = []
# Create dataset of 1/100th of the entries for quicker testing during development
for index in range(0,int((len) * (trainPerc/100))):
    if not (int(index/10) % 100):
        newsetIn.append(dataSet[index,:,1:10])
        newsetOut.append((dataSet[index,:,11]))
for idx in range(int((len) * (trainPerc/100)), len):
    if not (int(idx/10) % 10):
        testsetIn.append(dataSet[idx,:,1:10])
        testsetOut.append((dataSet[idx,:,11]))
testingIn = torch.Tensor(np.array(newsetIn)).float()
testingOut = torch.Tensor(np.array(newsetOut)).long()
inTest = torch.Tensor(np.array(testsetIn)).float()
outTest = torch.Tensor(np.array(testsetOut)).long()
# Create Dataloaders for all the datasets
dataLoaderTrain = data.DataLoader(data.TensorDataset(trainDataIn, trainDataOut), batch_size=batchSize, shuffle=False, num_workers=16, persistent_workers = True, drop_last= True)
dataLoaderTest = data.DataLoader(data.TensorDataset(testDataIn, testDataOut), batch_size=batchSize, shuffle=False, num_workers=16, persistent_workers = True, drop_last= True)
testingDataLoader = data.DataLoader(data.TensorDataset(testingIn, testingOut), batch_size=batchSize, shuffle = False, num_workers=16, persistent_workers = True, drop_last= True)
testingTestData = data.DataLoader(data.TensorDataset(testingIn, testingOut), batch_size=batchSize, shuffle = False, num_workers=16, persistent_workers = True, drop_last= True)

In [4]:
print(inTest.shape)
print(outTest.shape)
print()

torch.Size([19700, 10, 9])
torch.Size([19700, 10])



In [5]:
#TESTING
tom = np.array([[1,2,3,4],[9,8,7,6],[5,7,8,3],[9,8,7,3]])
print(tom)
tom = tom[np.argsort(tom[:, 2])]
print(tom)

[[1 2 3 4]
 [9 8 7 6]
 [5 7 8 3]
 [9 8 7 3]]
[[1 2 3 4]
 [9 8 7 6]
 [9 8 7 3]
 [5 7 8 3]]


In [61]:
# Creating Learner
class CfCLearner(pl.LightningModule):
    def __init__(self, model, lr):
        super().__init__()
        self.model = model
        self.lr = lr
        self.lossFunc = nn.CrossEntropyLoss()
    
    def training_step(self, batch, batch_idx):
        # Get in and out from batch
        inputs, target = batch
        # Put input through model
        output, _ = self.model.forward(inputs)
        # Reorganize inputs for use with loss function
        output = output.permute(0, 2, 1)
        # Calculate Loss using Cross Entropy Loss 
        loss = self.lossFunc(output, target)
        self.log("trainLoss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # Get in and out from batch
        inputs, target = batch
        # Put input through model
        output, _ = self.model.forward(inputs)
        # Reorganize inputs for use with loss function
        output = output.permute(0, 2, 1)
        print(f"output: {output.shape}")
        print(f"target: {target.shape}")
        # Calculate Loss using Cross Entropy Loss 
        loss = self.lossFunc(output, target)
        self.log("valLoss", loss, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self):
        # Using AdamW optomizer based on info from paper
        return torch.optim.AdamW(self.model.parameters(), lr = self.lr)

In [62]:
class Modena(nn.Module): 
    # CfC with feed-forward layer to classify at end.
    def __init__(self, inputSize, unitNum = None, motorNum = 2, outputDim = 2, batchFirst = True):
        super().__init__()
        if isinstance(inputSize, Modena):
            self.inputSize = inputSize.inputSize
            self.unitNum = inputSize.unitNum
            self.motorNum = inputSize.motorNum
            self.outputDim = inputSize.outputDim
            self.batchFirst = inputSize.batchFirst
            # Create NCP wiring for CfC
            wiring = AutoNCP(self.unitNum, self.motorNum)
            # Create CfC model with inputs and wiring
            self.cfc = CfC(self.inputSize, wiring, batch_first=self.batchFirst)
            # Create feed-forward layer
            self.fF = nn.Linear(self.motorNum, self.outputDim)
            self.fF.weight = nn.Parameter(inputSize.fF.weight)
        else:
            self.inputSize = inputSize
            self.unitNum = unitNum
            self.motorNum = motorNum
            self.outputDim = outputDim
            self.batchFirst = batchFirst
            # Create NCP wiring for CfC
            wiring = AutoNCP(unitNum, motorNum)
            # Create CfC model with inputs and wiring
            self.cfc = CfC(inputSize, wiring, batch_first=batchFirst)
            # Create feed-forward layer
            self.fF = nn.Linear(motorNum, outputDim)
        

    def forward(self, batch, hidden = None):
        batch, hidden = self.cfc(batch, hidden) # Pass inputs through CfC
        out = nn.functional.relu(self.fF(batch)) # pass through FeedForward Layer, then make 0 minimum
        return out, hidden # Return the guess and the hidden state

In [64]:
# OBU module class to organize
class OBU():
    def __init__(self, inputSize, units = 20, motors = 8, outputs = 20, epochs = 10, lr = 0.001, gpu = False):
        if isinstance(inputSize, OBU):
            self.lr = inputSize.lr
            self.epochs = inputSize.epochs
            self.gpu = inputSize.gpu
            self.model = Modena(inputSize.model)
            self.model.load_state_dict(inputSize.model.state_dict())
            self.learner = CfCLearner(self.model, self.lr)
            self.trainer = pl.Trainer(
                logger = CSVLogger('log'), # Set ouput destination of logs, logging accuracy every 50 steps
                max_epochs = self.epochs, # Number of epochs to train for
                gradient_clip_val = 1, # This is said to stabilize training, but we should test if that is true
                accelerator = "gpu" if self.gpu else "cpu" # Using the GPU to run training or not
                )
        else:
            self.lr = lr
            self.epochs = epochs
            self.gpu = gpu
            self.model = Modena(inputSize, units, motors, outputs)
            self.learner = CfCLearner(self.model, lr) # tune units, lr
            self.trainer = pl.Trainer(
                logger = CSVLogger('log'), # Set ouput destination of logs, logging accuracy every 50 steps
                max_epochs = epochs, # Number of epochs to train for
                gradient_clip_val = 1, # This is said to stabilize training, but we should test if that is true
                accelerator = "gpu" if gpu else "cpu" # Using the GPU to run training or not
                )
    
        # Overloading add function to create fed.avg. model
    def __add__(self, other):
        # if self.learner.getHidden() != None and other.learner.getHidden() != None:
        self.model.load_state_dict(dict( (n, self.model.state_dict().get(n, 0)+other.model.state_dict().get(n, 0)) for n in set(self.model.state_dict())|set(other.model.state_dict()) ))
        # elif other.learner.getHidden() != None:
        #     self.model.load_state_dict(other.model.state_dict())
        # elif self.learner.getHidden() != None:
        #     self.model.load_state_dict(self.model.state_dict())
        return self

    # Overloading div. function to average model
    def __truediv__(self, i):
        
        self.model.load_state_dict(dict((n, self.model.state_dict().get(n, 0)/i) for n in self.model.state_dict()))
        # self.model.load_state_dict(self.model.state_dict()/i)
        # self.learner.setHidden(self.learner.getHidden() / i)
        # self.model.fF.weight = nn.Parameter(self.model.fF.weight/i)
        return self
    
    def fit(self, dataLoader):
        # calling built in fit function
        return self.trainer.fit(self.learner, dataLoader)
    
    # Function to run model through a testing dataset and calculate accuracy. Can be expanded to give more metrics and more useful metrics.
    def test(self, dataIn, dataOut, extraLayer = True):
        # Put input data through model and determine classification
        with torch.no_grad():
            outs = self.model(dataIn)
        if extraLayer:
            outs = outs[0]
        outs = np.asarray(outs)
        outs = torch.from_numpy(outs)
        # Get the label with the maximum confidence for determining classification
        print(outs.shape)
        _, res = torch.max(outs, 2)
        countR = 0
        numZero = 0
        tot = outs.shape[0]
        total = 0
        for i in range(0, tot):
            # Loop through sequences of 10 each
            for t in range(0, res[i].shape[0]):
                # Loop through the sub-sequences
                if res[i,t] == dataOut[i,t]:
                    # Check if label is correct, and add to count right accordingly
                    countR += 1
                if dataOut[i,t] == 0:
                    # If the label is zero, increment the count of zeroes to determine if model is just outputting zeroes
                    numZero += 1
                total += 1
        # Calculate percent correct and percent zero
        perc = (countR/total) * 100
        percZero = (numZero/total) * 100
        print("Model got " + str(countR) + "/" + str(total) + " right. Accuracy of " + str(perc) + "%")
        print(str(percZero) + "% Zeroes.")
        return countR, total, perc, percZero
    
    def testStep(self, dataLoader):
        self.learner.validation_step(next(iter(dataLoader)), 0)
    
    def setModel(self, model):
        if not model == None:
            self.model = model

    def getModel(self):
        return self.model
    
    def resetTrainer(self):
        self.trainer = pl.Trainer(
            logger = CSVLogger('log'), # Set ouput destination of logs, logging accuracy every 50 steps
            max_epochs = self.epochs, # Number of epochs to train for
            gradient_clip_val = 1, # This is said to stabilize training, but we should test if that is true
            accelerator = "gpu" if self.gpu else "cpu" # Using the GPU to run training or not
            )


In [ ]:
# loop through split, and create model per vehicle.
# Run one epoch through each model, and average model and return.
# Continue loop for desired epochs

# With 2 epochs, 10 sub, and 0:3 models ending acc. of 0.1377%
# With 2 epochs, 10 sub, and 0:10 models ending acc. of
models = []
dataSets = []
results = []
subEpochs = 10
epochs = 2
gpu = False
# Create starting models
mainModel = OBU(9, epochs= subEpochs, gpu = gpu)
nextModel = OBU(9, epochs= subEpochs, gpu = gpu)
# Divide dataset of recieving vehicles among OBUs
for vehicle in splits[:50]:
    # Add new OBU for each model
    models.append(OBU(9, epochs = subEpochs, gpu=gpu))
    # Create Slice of dataset
    if vehicle.shape[0] < 32:
        print(vehicle.shape[0])
        vehicle = None
    else:
        vehicle = data.DataLoader(data.TensorDataset(vehicle[:,:,1:10].float(), vehicle[:,:,11].long()), batch_size=32, shuffle=False, num_workers=16, persistent_workers = True)
    # Add sub - dataset to dataset
    dataSets.append(vehicle)
# Train individual models and combine
for epoch in range(epochs):
    i = 0
    # Baseline model to add everything to. !!Do I want this or should it be a completely new model?!! Got 0% on combination before, testing with new model for next model.
    nextModel = OBU(9, epochs= subEpochs, gpu = gpu)
    # Train models
    for mod in models:
        # Make multithreaded?
        # set model to main model, and train that
        mod = OBU(mainModel)
        # Reset the trainer (should be unneeded now) to allow for further training
        # mod.resetTrainer()
        # Actually train
        if dataSets[i] != None:
            mod.fit(dataSets[i])
            # combine models
            nextModel = OBU(nextModel + mod)   
            _, _ , perc, _ = mod.test(inTest, outTest)
            # Test individual model
            results.append([epoch, i+1, perc])
            i+=1 
        else:
            print("SKIPPED {epoch}\n\n\n\n\n\n")
    # Create combined model
    
    mainModel = OBU(nextModel / i)
# Test combined model at end
_, _, perc, _ = mainModel.test(inTest, outTest)
results.append([-1, -1, perc])
print(results, file = open('results.txt', 'w'))


Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\will\miniconda3\envs\Kettering\Lib\site-packages\pytorch_lightning\trainer\setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 

11
26


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Using default `ModelCheckpoint`. Consider installing `litm

Epoch 4:  91%|█████████ | 10/11 [00:00<00:00, 41.43it/s, v_num=859, trainLoss=2.780]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [45]:
cumModel = hiddens[0][2]
iter = 1
for model in hiddens[1:]:
    if model[1] != -1:
        if iter == 0:
            cumModel = model[2]
        else:
            cumModel = cumModel + model[2]
        iter += 1
    else:
        print(model[0])
        print(cumModel/iter)
        print(model[2])
        print(cumModel/iter - model[2])
        iter = 0

0
tensor([[-0.9981, -1.0000,  1.0000, -0.8293, -1.0000, -1.0000,  1.0000, -1.0000,
          0.6075,  0.0442,  0.7163,  0.6767,  0.7933,  0.7151,  0.5629,  0.9533,
         -0.8753,  0.7856, -0.5999, -0.2247],
        [-0.9155, -1.0000,  1.0000, -0.7490, -1.0000, -1.0000,  1.0000, -1.0000,
          0.5863,  0.0659,  0.7266,  0.6708,  0.7904,  0.7140,  0.5576,  0.9531,
         -0.8754,  0.7839, -0.6045, -0.2218],
        [-0.9489, -1.0000,  1.0000, -0.8079, -1.0000, -1.0000,  1.0000, -1.0000,
          0.6044,  0.0083,  0.7259,  0.6747,  0.8049,  0.7217,  0.5656,  0.9527,
         -0.8770,  0.7812, -0.5921, -0.2208],
        [-0.9244, -1.0000,  1.0000, -0.8202, -1.0000, -1.0000,  1.0000, -1.0000,
          0.5903,  0.0450,  0.7244,  0.6746,  0.7943,  0.7130,  0.5605,  0.9540,
         -0.8758,  0.7817, -0.6040, -0.2236],
        [-0.8204, -1.0000,  1.0000, -0.8306, -1.0000, -1.0000,  1.0000, -1.0000,
          0.5760,  0.0688,  0.7257,  0.6738,  0.7918,  0.7142,  0.5560,  0.9537,
    

In [46]:
list = [0,1,2]
print(list[0:])

[0, 1, 2]


In [52]:
testOBU = OBU(
    inputSize = 9,  # Number of features per BSM
    units = 20, # Number of hidden cells
    motors = 8, # Number of motor neurons
    outputs = 20, # Number of possible labels
    epochs = 100,
    lr = 0.001,
    gpu = True
)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [59]:
testOBU.fit(dataSets[54])

IndexError: list index out of range

In [40]:
save = OBU(testOBU)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [57]:
_, _, perc, _ = testOBU.test(inTest, outTest)

torch.Size([19700, 10, 20])
Model got 114509/197000 right. Accuracy of 58.126395939086294%
41.08375634517766% Zeroes.


In [58]:
_, _, perc, _ = testOBU.test(testDataIn, testDataOut)

torch.Size([196990, 10, 20])
Model got 1159988/1969900 right. Accuracy of 58.88562871211737%
40.353165135286055% Zeroes.


In [42]:
_, _, perc, _ = save.test(inTest, outTest)

torch.Size([19700, 10, 20])
Model got 116065/197000 right. Accuracy of 58.91624365482233%
41.08375634517766% Zeroes.


In [24]:
print(testOBU.getHidden() - save.getHidden())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0

In [25]:
print(testOBU.model.fF.weight - save.model.fF.weight)

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]], grad_fn=<SubBackward0>)
